In [1]:
import json
from datetime import datetime , timezone
import time
import numpy as np
import codecs
import struct
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pycorrelate
import collections
import itertools
from mpl_toolkits import mplot3d
import cv2
import imutils
import os

pd.set_option('display.max_rows', 1000)
%matplotlib inline

### 1. Pick a session to work with!

In [2]:
metadata = pickle.load(open('./cleaned_data/metadata.p', 'rb'))
print('Please Choose a Session to Work With:')
for k in metadata: print(k, end=',  ')

Please Choose a Session to Work With:
david_2,  patrick_1,  patrick_2,  beata_1,  david_3b,  irmandy_1,  juliana_1,  irmandy_2,  patrick_3,  

In [118]:
SESSION = 'patrick_3'

let's print some basic summary statistics based on the metadata for this session:

In [119]:
s_meta = metadata[SESSION]

try:
    print(SESSION, 'on', s_meta['date'])
    print('-'*10)
    print(s_meta['glasses_sessions_total_duration_string'], 'hr of glassess data starting at', \
          s_meta['glasses_session_times'][0][0])
    print('%02d:%02d hr of    video data starting at ' % (s_meta['vid_duration_sec'] / 60 // 60, 
           s_meta['vid_duration_sec'] / 60 % 60 )  + s_meta['vid_start'])
    print('-'*10)
    print('glasses sessions:')
    for i in range(len(s_meta['glasses_session_times'])):
        print('session ' + str(i) + ':',s_meta['glasses_session_times'][i][0],'to',\
              s_meta['glasses_session_times'][i][1],'\tduration:%8.2f min' % s_meta['glasses_session_durations_min'][i])
except:
    print('failed to load video analysis file.')

patrick_3 on 04/28
----------
04:25 hr of glassess data starting at 04/28/21 01:10:27PM EDT
04:26 hr of    video data starting at 04/28/21 01:10:00PM EDT
----------
glasses sessions:
session 0: 04/28/21 01:10:27PM EDT to 04/28/21 05:35:58PM EDT 	duration:  265.52 min


In [120]:
vid_meta_data = pickle.load(open('./cleaned_data/' + SESSION + '/video_blink_sessions.p', "rb" ))

In [121]:
#HAND ANNOTATE BLINK VIDEO SNIPPETS
def string_to_timestamp_ms(datestring):
    return datetime.strptime(datestring, '%m/%d/%y %I:%M:%S%p %Z').timestamp()*1000

def timestamp_ms_to_string(timestamp):
    local_tz = datetime.now().astimezone().tzinfo
    return datetime.fromtimestamp(timestamp/1000, tz=local_tz).strftime('%m/%d/%y %I:%M:%S%p %Z')

def hand_annotate_frame(video_filename, video_start_timestamp_ms, start_timestamp_ms, end_timestamp_ms, ms_response_time=400, slow_down_ratio=1, fps=59.94):

    start_frame = int(((start_timestamp_ms - video_start_timestamp_ms)/1000)*fps)
    end_frame   = int(((end_timestamp_ms   - video_start_timestamp_ms)/1000)*fps)
  
    #blink times to frame list:
    stream_starttime_sec = video_start_timestamp_ms/1000. + start_frame/fps
    frame_duration_sec = 1/fps #time between frames in sec
    
    if slow_down_ratio == 1:
        ms_delay = 1
    else:
        ms_delay = int(slow_down_ratio*(1/60)*1000) #map 2 to twice as slow as 60Hz delay in ms, 3 as 3x, etc
     
    #======================================================================================
    #loop through, if 'b' pressed add a blinktime at that frame/time - ms_response,
    #include some time after the video is over in case a blink happens at the end.
    
    
    while True:
        
        blink_times = []
        
        WINDOW_NAME = 'blink_window'
        cv2.namedWindow(WINDOW_NAME, 2*cv2.WINDOW_AUTOSIZE)
        cv2.startWindowThread()

        vs = cv2.VideoCapture(video_filename)
        vs.set(cv2.CAP_PROP_POS_FRAMES, start_frame-1)
        time.sleep(1.0)


        prev_frame, blink_count = 0, 0
        for fno in range(0, end_frame-start_frame):
            _, frame = vs.read()  

            # show the frame
            cv2.imshow(WINDOW_NAME, frame)
            
            if not fno:
                os.system('''/usr/bin/osascript -e 'tell app "Finder" to set frontmost of process "Python" to true' ''') 
    
            
            key = cv2.waitKey(ms_delay) & 0xFF

            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break
            if key == ord("b"):
                blink_times.append(((fno/60)*1000) + start_timestamp_ms - ms_response_time)
                
        start = time.time()*1000.0
        if cv2.waitKey(ms_response_time) != -1:
            end = time.time()*1000.0
            blink_times.append((end-start) + end_timestamp_ms - ms_response_time)
        
        print('got blink times of', blink_times)
        print('now playing back to show the alignment...')
        #========================================================================#
        #now play it back with those times we input overlay

        blink_sig = np.zeros(end_frame-start_frame)
        blink_dur_sec = 0.5
        blink_dur_sample = int(blink_dur_sec*fps)
        
        #whenever we have a blink, put ones in there for blink_dur frames.  Make sure the sample before that blink is zero
        for btime in blink_times:
            btime_sec_stream = (btime/1000.)-stream_starttime_sec
            bframe = int(btime_sec_stream*fps)
            
            if bframe >=0 and (bframe + blink_dur_sample) < len(blink_sig):
                blink_sig[bframe-1] = 0  
                blink_sig[bframe:bframe+blink_dur_sample] = 1
            elif bframe < 1 and bframe + blink_dur_sample > 0:
                try:
                    blink_sig[0:bframe+blink_dur_sample] = 1
                except:
                    blink_sig[0:-1] = 1
            elif bframe + blink_dur_sample > len(blink_sig) and bframe < (end_frame-start_frame):
                blink_sig[bframe-1] = 0  
                blink_sig[bframe:-1] = 1
                
        #reset the frame to the start
        vs.set(cv2.CAP_PROP_POS_FRAMES, start_frame-1)
        time.sleep(1.0)

        prev_frame, blink_count = 0, 0
        for fno in range(0, end_frame-start_frame):
            _, frame = vs.read()  

            if blink_sig[fno] and not prev_frame:
                blink_count += 1

            cv2.putText(frame, "COUNT: %d"%(blink_count), (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            if blink_sig[fno]:
                cv2.putText(frame, "BLINKING NOW!!!!!!", (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # show the frame
            cv2.imshow(WINDOW_NAME, frame)
            key = cv2.waitKey(1) & 0xFF

            prev_frame = blink_sig[fno]

            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break

        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        
        os.system('''/usr/bin/osascript -e 'tell app "Finder" to set frontmost of process "Chrome" to true' ''') 

        #now either accept, skip, bad_track, or redo
        assess = input('Please give 1 for accept, 2 for redo, 3 for skip')
        assess = int(assess)
        if assess == 1:
            return blink_times
        if assess == 3:
            return None
    
    
    
    
#======= HAND LABEL TOOL ========#
# 1. Loop through blink video sessions
# 2. For each, open up and play the video snippet, get 'b' keystrokes, make a df
# 3. Play back with 'b' keystrokes overlaid
# 4. accept, redo, skip, no face
# each snippet will now be labeled as 'no face', 'skipped', or [array of blinktimes hand labeled]

In [122]:
indices = (-np.array(vid_meta_data['duration_sessions_sec'])).argsort()

In [123]:
hand_annotations = {}

In [125]:
try:
    hand_annotations = pickle.load(open('./cleaned_data/' + SESSION + '/hand_annotations.p', "rb" ))
except:
    print('no hand annotations')

In [137]:
vid_start_timestamp = string_to_timestamp_ms(s_meta['vid_start'])

#did 20:27, 2:4, juliana1
#did 6:8, 20:25, 30:47,  beata1
# 45:60, irmandy2
# 40:75, david3b
# 30:
for sess in indices[37:40]:
    if sess not in hand_annotations:
        hand_annotations[sess] = hand_annotate_frame('/Volumes/ExtDrive_(ResEnv)/data_cap_val_2/' + SESSION + '/video_crop_1.mp4',  #/openface/video_adobe.mp4
         #hand_annotations[sess] = hand_annotate_frame('/Volumes/ExtDrive_(ResEnv)/data_cap_val_2/david_3/video_part_2.mp4',  #/openface/video_adobe.mp4
                                                      vid_start_timestamp,  
                                                      vid_meta_data['session_times'][sess][0],
                                                      vid_meta_data['session_times'][sess][1],
                                                     slow_down_ratio=1,
                                                      ms_response_time=400)
            
            
            
print(hand_annotations)

got blink times of [1619641528482.0, 1619641536798.6667]
now playing back to show the alignment...


Please give 1 for accept, 2 for redo, 3 for skip 2


got blink times of [1619641528482.0, 1619641536782.0, 1619641537598.6667, 1619641555998.6667, 1619641556715.3333, 1619641569482.0, 1619641571932.0, 1619641572748.6667, 1619641576715.3333, 1619641577298.6667, 1619641578948.6667, 1619641583298.6667, 1619641584815.3333, 1619641586815.3333, 1619641589182.0, 1619641590665.3333, 1619641592198.6667, 1619641592832.0, 1619641595565.3333, 1619641596882.0, 1619641599165.3333, 1619641601665.3333, 1619641606948.6667, 1619641609782.0, 1619641611398.6667, 1619641612482.0, 1619641613848.6667, 1619641616582.0, 1619641617715.3333, 1619641618898.6667, 1619641619498.6667, 1619641621615.3333, 1619641622482.0, 1619641623515.3333, 1619641624182.0, 1619641626565.3333, 1619641627065.3333, 1619641628498.6667]
now playing back to show the alignment...


Please give 1 for accept, 2 for redo, 3 for skip 1


got blink times of [1619644547949.0, 1619644548549.0, 1619644549782.3333, 1619644564915.6667, 1619644573682.3333, 1619644588549.0, 1619644589082.3333, 1619644596732.3333, 1619644606465.6667, 1619644606932.3333, 1619644624432.3333, 1619644631765.6667]
now playing back to show the alignment...


Please give 1 for accept, 2 for redo, 3 for skip 1


got blink times of [1619635406995.6667, 1619635414479.0, 1619635420562.3333, 1619635434979.0, 1619635439729.0, 1619635456129.0, 1619635456995.6667, 1619635459279.0, 1619635459862.3333, 1619635460362.3333, 1619635463729.0, 1619635464479.0, 1619635467879.0, 1619635469129.0, 1619635471062.3333, 1619635473229.0, 1619635476062.3333, 1619635476929.0, 1619635477462.3333, 1619635481095.6667, 1619635485595.6667, 1619635488112.3333, 1619635489045.6667, 1619635489712.3333, 1619635491729.0, 1619635492229.0, 1619635492545.6667, 1619635492912.3333, 1619635493827.164]
now playing back to show the alignment...


Please give 1 for accept, 2 for redo, 3 for skip 1


{5: [1619629819285.0, 1619629819485.0, 1619629820368.3333], 6: [1619629825039.3333, 1619629826856.0, 1619629836539.3333, 1619629848771.782], 7: [1619629854937.6667, 1619629858371.0, 1619629864271.0, 1619629866054.3333, 1619629866954.3333, 1619629870587.6667, 1619629872387.6667], 8: [1619629875239.3333, 1619629875956.0, 1619629876206.0, 1619629876406.0], 9: [], 10: [1619629882030.6667, 1619629882547.3333], 11: [], 12: [1619629888656.998], 13: [1619629894643.3333, 1619629897026.6667], 14: None, 15: [1619630098480.6667, 1619630098847.3333, 1619630100264.0, 1619630101397.3333, 1619630101964.0, 1619630102780.6667, 1619630104664.0, 1619630107347.3333, 1619630108697.3333, 1619630109897.3333, 1619630112814.0, 1619630113780.6667, 1619630113947.3333, 1619630115397.3333, 1619630115684.7502], 17: [1619630148747.0, 1619630149656.948], 18: [1619630151016.6667, 1619630153000.0, 1619630155183.3333, 1619630157416.6667, 1619630159716.6667, 1619630161416.6667, 1619630163633.3333, 1619630164116.6667, 1619

In [138]:
pickle.dump(hand_annotations, open( './cleaned_data/' + SESSION + '/hand_annotations.p', "wb" ))

In [139]:
total=0
for k in hand_annotations: 
    try:
        total+=len(hand_annotations[k])
    except:
        pass
print(total)

544
